# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29


You should consider upgrading via the 'C:\Users\brics\Documents\git_repos\git_repos\lang-chain\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.environ['OPENAI_MODEL_NAME'] = os.getenv('OPENAI_API_KEY')
os.environ['SERPER_API_KEY'] = os.getenv('SERPER_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


In [6]:
from crewai import Agent, Crew, Task

## crewAI Tools

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [8]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [9]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [10]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [11]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
# strong typed data
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [12]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [13]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [14]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [15]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [17]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I should consider the theme, size, and budget constraints when selecting the venue.

Action:
Search the internet

Action Input:
{
  "search_query": "tech conference venues in San Francisco"
}
 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: Running for over twenty years, Dreamforce -- Salesforce's flagship annual conference -- is widely known as one of the biggest tech conferences ...
---
Title: Discover Tech Conferences Events & Activities in San Francisco, CA
Link: https://www.eventbrite.com/d/ca--san-francisco/conferences/tech-conferences/
Snippet: Tech conferences

I need to effectively market the Tech Innovation Conference and engage at least 500 potential attendees. I should utilize all available tools to gather information and maximize event exposure.

Action:
Search the internet

Action Input:
{
  "search_query": "Tech Innovation Conference marketing strategies"
}
 


Search results: Title: Innovative Event Marketing Strategies to Boost Corporate and Social ...
Link: https://www.linkedin.com/pulse/innovative-event-marketing-strategies-boost-corporate-b70rc
Snippet: These strategies are designed to attract, entertain, and leave a lasting impression on attendees even after the event is over. From intriguing ...
---
Title: 10 Effective Conference Marketing Ideas for 2024 | InEvent
Link: https://inevent.com/blog/others/conference-marketing-ideas.html
Snippet: 1. Focus on Target Audience and Personalization · 2. Leverage Digital Marketing for Conference Advertising · 3. Create a Stunning Event Website · 4 ...
---
Title: 2024 Digital Marketing Stra

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'South Beach, San Francisco, CA',
 'booking_status': 'Whole Venue for $375 per hour',
 'capacity': 110,
 'name': 'American Bookbinders Museum'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [19]:
from IPython.display import Markdown
Markdown("marketing_report.md")

### Marketing Activities and Attendee Engagement Report

To promote the Tech Innovation Conference and engage with potential attendees effectively, we will utilize a combination of innovative marketing strategies and ideas. Here is a detailed plan of action based on the research conducted:

1. **Tap into Industry Influencers**: Identify top influencers in the tech industry and engage them to promote the event to their followers.

2. **Offer Attendee Incentives**: Provide incentives and discounts to encourage new attendees and group bookings.

3. **Utilize Speakers and Sponsors**: Equip speakers and sponsors with marketing packs to help promote their involvement in the conference.

4. **Create Video Content**: Develop video content such as testimonials, past conference highlights, and personal invitations to enhance engagement.

5. **Leverage Social Media**: Establish dedicated social media accounts, create groups, and run targeted advertising campaigns to expand reach and engagement.

6. **Find Common Ground**: Collaborate with organizations, events, and communities that share a similar target audience to amplify promotional efforts.

7. **Utilize Email Marketing**: Incorporate the conference logo or banner in email signatures to increase visibility.

8. **Enable Social Media Sharing**: Add share buttons to post-purchase pages to encourage attendees to promote the event to their networks.

9. **Implement Remarketing**: Consider running a remarketing campaign through Google Ads to re-engage with interested audiences.

By implementing these strategies and ideas, we aim to enhance event exposure, increase attendee engagement, and achieve our goal of attracting at least 500 potential attendees to the Tech Innovation Conference.